In [160]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

In [161]:
df = pd.read_csv('diabetes.csv')
# iris = load_iris()
# df = pd.DataFrame(iris.data, columns=iris.feature_names)
# df['target'] = iris.target
df.head(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [162]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [163]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)
X = pd.DataFrame(X_norm, columns=X.columns)
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000


In [164]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35)

In [165]:
y_prob = (pd.value_counts(y_train) / len(y_train))
y_prob

0    0.629259
1    0.370741
Name: Outcome, dtype: float64

In [166]:
def meanStd(X_train, y_train):
    mean_data = pd.DataFrame(np.empty((len(y_prob), X_train.shape[1])) ,index=list(y_prob.index) , columns=list(X_train))
    std_data = mean_data.copy()
    for i in y_prob.index:
        pXi = X_train[y_train == i]
        mean_data.loc[i] = pXi.mean(axis=0)
        std_data.loc[i] = pXi.std(axis=0)
    return mean_data, std_data   

In [167]:
mean_data, std_data = meanStd(X_train, y_train)
mean_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.193893,0.551532,0.552156,0.190021,0.079588,0.451066,0.146128,0.169851
1,0.282035,0.714030,0.566681,0.239421,0.132931,0.522810,0.200679,0.267207


In [168]:
def gnb(X, currentClass, idx):
    X_std = std_data.loc[currentClass, idx]
    X_mean = mean_data.loc[currentClass, idx]
    X_var = X_std ** 2
    nr = np.exp(-((X - X_mean) ** 2) / (2 * X_var))
    dr = np.sqrt(np.sqrt(2 * np.pi) * X_std)
    return nr / dr

In [169]:
def predict(X_test):
    # prob = np.empty(len(y_prob))
    prob = pd.Series(np.empty(len(y_prob)), index=list(y_prob.index))
    for i in y_prob.index:
        pClass = y_prob[i]
        pPred = pClass
        for k, v in X_test.iteritems():
            pPred *= gnb(v, i, k)
        prob[i] = pPred
    return prob.idxmax()

In [170]:
from sklearn.metrics import accuracy_score
y_pred = X_test.apply(predict, axis=1)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7546468401486989